In [5]:
# Exclusivo para google colab, não necessário caso já estejam instaladas na máquina as bibliotecas necessárias

!pip install ltn
!pip install torch torchvision
!pip install datasets transformers

In [6]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets
import numpy as np
import random

# ======= Semente para reprodutibilidade =======
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

# =====================================================================
# 1. DATASET CIFAR-10 (APENAS CÃES E GATOS)
# =====================================================================

class CIFAR10DogsAndCats(Dataset):
    """
    Dataset customizado que carrega CIFAR-10 e filtra apenas
    as classes de cães (classe 5) e gatos (classe 3).
    """
    def __init__(self, train=True, transform=None, download=True):
        print("📥 Carregando CIFAR-10...")

        # Baixar e carregar CIFAR-10
        self.cifar10 = datasets.CIFAR10(
            root='./data',
            train=train,
            download=download,
            transform=None  # Aplicaremos transformação depois
        )

        # Índices das classes: 3=gato, 5=cão
        self.dog_class = 5
        self.cat_class = 3

        # Filtrar apenas cães e gatos
        self.dog_indices = [i for i, (_, label) in enumerate(self.cifar10) if label == self.dog_class]
        self.cat_indices = [i for i, (_, label) in enumerate(self.cifar10) if label == self.cat_class]

        # Embaralhar
        random.shuffle(self.dog_indices)
        random.shuffle(self.cat_indices)

        self.transform = transform
        self.dataset_type = 'train' if train else 'test'

        print(f"✓ Cães encontrados: {len(self.dog_indices)}")
        print(f"✓ Gatos encontrados: {len(self.cat_indices)}")
        print(f"✓ Pares balanceados: {min(len(self.dog_indices), len(self.cat_indices))}\n")

    def __len__(self):
        return min(len(self.dog_indices), len(self.cat_indices))

    def __getitem__(self, idx):
        # Obter índices dos pares balanceados
        dog_idx = self.dog_indices[idx % len(self.dog_indices)]
        cat_idx = self.cat_indices[idx % len(self.cat_indices)]

        # Carregar imagens do CIFAR-10
        dog_img, _ = self.cifar10[dog_idx]
        cat_img, _ = self.cifar10[cat_idx]

        # Aplicar transformações
        if self.transform:
            dog_img = self.transform(dog_img)
            cat_img = self.transform(cat_img)

        return dog_img, cat_img


def get_transforms(image_size=32, augmentation=True):
    """Define transformações para as imagens CIFAR-10"""
    if augmentation:
        return transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomRotation(8),
            transforms.ColorJitter(brightness=0.15, contrast=0.15, saturation=0.15),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    else:
        return transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])


def create_dataloader(batch_size=64, num_workers=0, augmentation=True, train=True):
    """
    Cria DataLoader para CIFAR-10 (apenas cães e gatos)

    Args:
        batch_size: tamanho do batch
        num_workers: número de workers para carregamento
        augmentation: aplicar data augmentation
        train: usar dataset de treino ou teste
    """
    transform = get_transforms(augmentation=augmentation)
    dataset = CIFAR10DogsAndCats(train=train, transform=transform, download=True)

    if len(dataset) == 0:
        raise ValueError("Dataset está vazio!")

    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True if train else False,
        num_workers=num_workers,
        pin_memory=True if torch.cuda.is_available() else False
    )
    print(f"✓ DataLoader criado com {len(dataloader)} batches\n")
    return dataloader


# =====================================================================
# 2. MODELO CNN
# =====================================================================

class CNNModel(nn.Module):
    """
    CNN otimizado para imagens 32x32 do CIFAR-10
    """
    def __init__(self, input_channels=3, num_classes=1):
        super(CNNModel, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(input_channels, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.conv3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((2, 2))
        )

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 2 * 2, 256),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)  # logits (sem Sigmoid)
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.fc(x)
        return x  # logits


# =====================================================================
# 3. TREINAMENTO COM LTN
# =====================================================================

def train_ltn_classifier(n_epochs=10, batch_size=64, learning_rate=0.001, device=None):
    """
    Treinamento com BCEWithLogits + LTN fuzzy
    """
    device = device or (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))
    print(f"🖥️  Usando device: {device}\n")

    # DataLoader
    print("=" * 60)
    print("ETAPA 1: Carregando Dataset CIFAR-10")
    print("=" * 60)
    train_dataloader = create_dataloader(batch_size=batch_size, augmentation=True, train=True)
    test_dataloader = create_dataloader(batch_size=batch_size, augmentation=False, train=False)

    # Modelo
    print("=" * 60)
    print("ETAPA 2: Criando Modelo CNN")
    print("=" * 60)
    model = CNNModel(input_channels=3, num_classes=1)
    model = model.to(device)
    print(f"✓ Modelo criado com {sum(p.numel() for p in model.parameters())} parâmetros\n")

    # Otimizador e scheduler
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

    # BCEWithLogitsLoss
    pos_weight = torch.tensor([1.5]).to(device)
    bce_loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

    best_loss = float('inf')

    print("=" * 60)
    print("ETAPA 3: Iniciando Treinamento")
    print("=" * 60 + "\n")

    for epoch in range(n_epochs):
        model.train()
        epoch_loss = 0.0
        correct_dogs = 0
        correct_cats = 0
        total_samples = 0

        for i, (dog_imgs, cat_imgs) in enumerate(train_dataloader):
            dog_imgs = dog_imgs.to(device)
            cat_imgs = cat_imgs.to(device)

            # Concatena cães + gatos
            inputs = torch.cat([dog_imgs, cat_imgs], dim=0)
            labels = torch.cat([
                torch.ones(dog_imgs.size(0), 1, device=device),
                torch.zeros(cat_imgs.size(0), 1, device=device)
            ], dim=0)

            optimizer.zero_grad()

            logits = model(inputs)

            # LTN fuzzy
            probs = torch.sigmoid(logits)
            dogs_probs = probs[:dog_imgs.size(0)]
            cats_probs = probs[dog_imgs.size(0):]
            phi1 = torch.mean(dogs_probs)
            phi2 = torch.mean(1.0 - cats_probs)
            sat_agg = (phi1 + phi2) / 2.0
            ltn_loss = 1.0 - sat_agg

            # BCEWithLogitsLoss
            bce_total = bce_loss_fn(logits, labels)

            # Combinar losses
            loss = 0.4 * ltn_loss + 0.6 * bce_total

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            epoch_loss += loss.item()

            # Métricas
            with torch.no_grad():
                preds = (probs > 0.5).float()
                correct_dogs += preds[:dog_imgs.size(0)].sum().item()
                correct_cats += (1 - preds[dog_imgs.size(0):]).sum().item()
                total_samples += inputs.size(0)

            if (i + 1) % 5 == 0 or (i + 1) == len(train_dataloader):
                batch_acc = (correct_dogs + correct_cats) / total_samples * 100
                print(f"Epoch [{epoch+1}/{n_epochs}], Step [{i+1}/{len(train_dataloader)}], "
                      f"Loss: {loss.item():.4f}, Acc: {batch_acc:.2f}%")

        epoch_loss = epoch_loss / len(train_dataloader)
        epoch_acc = (correct_dogs + correct_cats) / total_samples * 100

        scheduler.step(epoch_loss)

        print(f"\n{'='*60}")
        print(f"Epoch [{epoch+1}/{n_epochs}] Completado")
        print(f"Loss Médio: {epoch_loss:.4f}")
        print(f"Acurácia: {epoch_acc:.2f}%")
        print(f"{'='*60}\n")

        if epoch_loss < best_loss:
            best_loss = epoch_loss
            print(f"💾 Novo melhor modelo. Loss: {best_loss:.4f}\n")

    print("🎉 Treinamento concluído!\n")
    return model, train_dataloader, test_dataloader


# =====================================================================
# 4. AVALIAÇÃO
# =====================================================================

def evaluate_model(model, dataloader, device, num_samples=5):
    model.eval()
    print("=" * 60)
    print("AVALIAÇÃO DO MODELO")
    print("=" * 60)

    with torch.no_grad():
        dog_imgs, cat_imgs = next(iter(dataloader))
        dog_imgs = dog_imgs[:num_samples].to(device)
        cat_imgs = cat_imgs[:num_samples].to(device)

        inputs = torch.cat([dog_imgs, cat_imgs], dim=0)
        logits = model(inputs)
        probs = torch.sigmoid(logits)

        dog_probs = probs[:num_samples]
        cat_probs = probs[num_samples:]

        print(f"\nPredições para {num_samples} imagens de CÃES:")
        for i, p in enumerate(dog_probs):
            print(f"  Imagem {i+1}: {p.item():.4f} (esperado: ~1.0)")

        print(f"\nPredições para {num_samples} imagens de GATOS:")
        for i, p in enumerate(cat_probs):
            print(f"  Imagem {i+1}: {p.item():.4f} (esperado: ~0.0)")

        dog_correct = (dog_probs > 0.5).sum().item()
        cat_correct = (cat_probs <= 0.5).sum().item()
        accuracy = (dog_correct + cat_correct) / (2 * num_samples) * 100

        print(f"\n✓ Acurácia nas amostras: {accuracy:.2f}%")
        print("=" * 60 + "\n")


# =====================================================================
# 5. EXECUÇÃO PRINCIPAL
# =====================================================================

if __name__ == "__main__":
    print("\n" + "=" * 60)
    print("CLASSIFICADOR BINÁRIO: CÃES vs GATOS (CIFAR-10)")
    print("Usando BCEWithLogits + Lógica fuzzy (LTN simplificada)")
    print("=" * 60 + "\n")

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    model, train_dataloader, test_dataloader = train_ltn_classifier(
        n_epochs=10,
        batch_size=64,
        learning_rate=0.001,
        device=device
    )

    # Avaliação
    evaluate_model(model, test_dataloader, device, num_samples=5)


CLASSIFICADOR BINÁRIO: CÃES vs GATOS (CIFAR-10)
Usando BCEWithLogits + Lógica fuzzy (LTN simplificada)

🖥️  Usando device: cuda

ETAPA 1: Carregando Dataset CIFAR-10
📥 Carregando CIFAR-10...
✓ Cães encontrados: 5000
✓ Gatos encontrados: 5000
✓ Pares balanceados: 5000

✓ DataLoader criado com 79 batches

📥 Carregando CIFAR-10...
✓ Cães encontrados: 1000
✓ Gatos encontrados: 1000
✓ Pares balanceados: 1000

✓ DataLoader criado com 16 batches

ETAPA 2: Criando Modelo CNN
✓ Modelo criado com 1442625 parâmetros

ETAPA 3: Iniciando Treinamento

Epoch [1/10], Step [5/79], Loss: 0.7140, Acc: 50.62%
Epoch [1/10], Step [10/79], Loss: 0.6887, Acc: 51.64%
Epoch [1/10], Step [15/79], Loss: 0.6676, Acc: 54.37%
Epoch [1/10], Step [20/79], Loss: 0.6727, Acc: 55.59%
Epoch [1/10], Step [25/79], Loss: 0.6701, Acc: 57.72%
Epoch [1/10], Step [30/79], Loss: 0.5450, Acc: 59.32%
Epoch [1/10], Step [35/79], Loss: 0.5757, Acc: 60.38%
Epoch [1/10], Step [40/79], Loss: 0.6304, Acc: 60.94%
Epoch [1/10], Step [45/7